In [13]:
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from datetime import datetime

In [14]:
#https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [9]:
# load data
def parse(x):
    return datetime.strptime(x, '%Y %m %d %H')

dataset = read_csv('raw.csv',  parse_dates = [['year', 'month', 'day', 'hour']], index_col=0, date_parser=parse)
dataset.drop('No', axis=1, inplace=True)
# manually specify column names
dataset.columns = ['pollution', 'dew', 'temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain']
dataset.index.name = 'date'
# mark all NA values with 0
dataset['pollution'].fillna(0, inplace=True)
# drop the first 24 hours
dataset = dataset[24:]
# summarize first 5 rows
print(dataset.head(5))
# save to file
dataset.to_csv('pollution.csv')

                     pollution  dew  temp   press wnd_dir  wnd_spd  snow  rain
date                                                                          
2010-01-02 00:00:00      129.0  -16  -4.0  1020.0      SE     1.79     0     0
2010-01-02 01:00:00      148.0  -15  -4.0  1020.0      SE     2.68     0     0
2010-01-02 02:00:00      159.0  -11  -5.0  1021.0      SE     3.57     0     0
2010-01-02 03:00:00      181.0   -7  -5.0  1022.0      SE     5.36     1     0
2010-01-02 04:00:00      138.0   -7  -5.0  1022.0      SE     6.25     2     0


In [19]:
reframed = series_to_supervised(dataset, 1, 2)

In [20]:
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var1(t),var2(t),...,var7(t),var8(t),var1(t+1),var2(t+1),var3(t+1),var4(t+1),var5(t+1),var6(t+1),var7(t+1),var8(t+1)
date,,,,,,,,,,,,,,,,,,,,,
2010-01-02 01:00:00,129.0,-16.0,-4.0,1020.0,SE,1.79,0.0,0.0,148.0,-15,...,0,0,159.0,-11.0,-5.0,1021.0,SE,3.57,0.0,0.0
2010-01-02 02:00:00,148.0,-15.0,-4.0,1020.0,SE,2.68,0.0,0.0,159.0,-11,...,0,0,181.0,-7.0,-5.0,1022.0,SE,5.36,1.0,0.0
2010-01-02 03:00:00,159.0,-11.0,-5.0,1021.0,SE,3.57,0.0,0.0,181.0,-7,...,1,0,138.0,-7.0,-5.0,1022.0,SE,6.25,2.0,0.0
2010-01-02 04:00:00,181.0,-7.0,-5.0,1022.0,SE,5.36,1.0,0.0,138.0,-7,...,2,0,109.0,-7.0,-6.0,1022.0,SE,7.14,3.0,0.0
2010-01-02 05:00:00,138.0,-7.0,-5.0,1022.0,SE,6.25,2.0,0.0,109.0,-7,...,3,0,105.0,-7.0,-6.0,1023.0,SE,8.93,4.0,0.0
